## Тема 3.2 Выборка данных из одной таблицы.

## Описание исходных таблиц

Supabase: https://supabase.com/

В примерах используются три таблицы, каждая отражает разную сущность:

### 1. **users**  
**user_id** (PK) — уникальный идентификатор пользователя  
**age** — возраст пользователя  
**churn_date** — дата ухода пользователя (если ушёл, иначе NULL)  
**city** — город проживания  
**first_name** — имя  
**last_name** — фамилия  
**reg_date** — дата регистрации  
**tariff** — используемый тариф

### 2. **internet**  
**id** (PK) — уникальный идентификатор интернет-сессии  
**mb_used** — количество мегабайт, потраченных за сессию  
**session_date** — дата использования  
**user_id** — идентификатор пользователя, совершившего сессию (FK на users.user_id)

### 3. **messages**  
**id** (PK) — уникальный идентификатор сообщения  
**message_date** — дата отправки сообщения  
**user_id** — идентификатор пользователя, отправившего сообщение (FK на users.user_id)

---

# 1. Виды JOIN в PostgreSQL + Примеры

**JOIN** — оператор для объединения данных из двух и более таблиц по связующему полю (например, user_id). Основные виды JOIN:

| JOIN         | Описание                                                       |
|--------------|----------------------------------------------------------------|
| INNER JOIN   | Только строки с совпадениями в обеих таблицах                  |
| LEFT JOIN    | Все из левой таблицы + совпадения из правой (если есть)        |
| RIGHT JOIN   | Все из правой таблицы + совпадения из левой (если есть)        |
| FULL JOIN    | Все строки из обеих таблиц                                     |
| CROSS JOIN   | Каждая строка левой комбинируется с каждой строкой правой      |

---

### 1.1 INNER JOIN

**Задача:** Показать только интернет-сессии с существующими пользователями.

```sql
SELECT 
  internet.id,
  internet.session_date,
  users.first_name,
  users.last_name
FROM 
  internet
INNER JOIN users ON internet.user_id = users.user_id;
```
**Результат:** Только те сессии, user_id которых есть в users.

---

### 1.2 LEFT JOIN

**Задача:** Посмотреть всех пользователей и их расход трафика. Если не пользовался интернетом — будет NULL.

```sql
SELECT
  users.user_id,
  users.first_name,
  internet.mb_used
FROM
  users
LEFT JOIN internet ON users.user_id = internet.user_id;
```
**Результат:** Показывает всех пользователей, даже если у них нет интернет-сессий.

---

### 1.3 RIGHT JOIN

**Задача:** Посмотреть все интернет-сессии и имена пользователей (если есть). Незарегистрированные user_id — NULL.

```sql
SELECT
  internet.id,
  internet.user_id,
  users.first_name
FROM
  users
RIGHT JOIN internet ON users.user_id = internet.user_id;
```
**Результат:** Показывает все интернет-сессии, даже если связанных сессий в users нет.

---

### 1.4 FULL JOIN

**Задача:** Показать все пользователи и сессии, даже если нет совпадений.

```sql
SELECT
  users.user_id,
  users.first_name,
  internet.id AS session_id,
  internet.mb_used
FROM
  users
FULL JOIN internet ON users.user_id = internet.user_id;
```
**Результат:** Отображает все данные обеих таблиц; если нет совпадений — NULL.

---

### 1.5 CROSS JOIN

**Задача:** Каждая пара пользователь – интернет-сессия.

```sql
SELECT
  users.first_name,
  internet.mb_used
FROM
  users
CROSS JOIN internet;
```
**Результат:** Все возможные комбинации пользователей и интернет-сессий.

---

# 2. UNION ALL

**UNION ALL** — объединяет результаты двух SELECT-запросов и сохраняет дубликаты (в отличие от UNION).

### Пример 1: Все user_id, связанные к интернету или сообщениям

```sql
SELECT user_id FROM internet
UNION ALL
SELECT user_id FROM messages;
```
**Результат:** user_id встречается сколько раз есть в обеих таблицах.

---

### Пример 2: Список событий пользователей (интернет-сессии и сообщения) — дата и id пользователя

```sql
SELECT user_id, session_date AS event_date FROM internet
UNION ALL
SELECT user_id, message_date AS event_date FROM messages;
```
**Результат:** Хронологический список действий: интернет-сессии и сообщения.

---

# 3. GROUP BY

**GROUP BY** — группирует строки по одному/нескольким столбцам и позволяет использовать агрегатные функции (COUNT, SUM, AVG...).

### Пример 1: Сообщения по пользователям

```sql
SELECT
  user_id,
  COUNT(*) AS messages_count
FROM
  messages
GROUP BY
  user_id;
```
**Результат:** Для каждого пользователя — сколько сообщений он отправил.

---

### Пример 2: Количество пользователей в каждом городе

```sql
SELECT
  city,
  COUNT(*) AS user_count
FROM
  users
GROUP BY
  city;
```
**Результат:** Для каждого города — сколько зарегистрировано пользователей.

---

### Пример 3: Сессии по датам для каждого пользователя

```sql
SELECT
  user_id,
  session_date,
  COUNT(*) AS sessions_per_day
FROM
  internet
GROUP BY
  user_id, session_date;
```
**Результат:** Сколько сессий у каждого пользователя за каждый день.

---

# 4. HAVING

**HAVING** — фильтрует группы после GROUP BY. Для агрегатных условий (например, “где пользователей больше 10”).

### Пример 1: Только города с более чем 5 пользователями

```sql
SELECT
  city,
  COUNT(*) AS users_in_city
FROM
  users
GROUP BY
  city
HAVING
  COUNT(*) > 5;
```
**Результат:** Только те города, где зарегистрировано больше 5 пользователей.

---

### Пример 2: Пользователи, которые отправляли более 10 сообщений

```sql
SELECT
  user_id,
  COUNT(*) AS messages_count
FROM
  messages
GROUP BY
  user_id
HAVING
  COUNT(*) > 10;
```
**Результат:** Выведет только тех пользователей, кто отправил более 10 сообщений.

---

### Пример 3: Пользователи, которые в каком-либо дне отправляли более одного сообщения

```sql
SELECT
  user_id,
  message_date,
  COUNT(*) AS msgs_per_day
FROM
  messages
GROUP BY
  user_id, message_date
HAVING
  COUNT(*) > 1;
```
**Результат:** Будут показаны только те дни/пользователи, где было отправлено больше 1 сообщения.

---

# 5. Комбинированный пример

**Задача:** Для каждого пользователя узнать, сколько у него было интернет-сессий и сколько сообщений.

```sql
SELECT
  u.user_id,
  u.first_name,
  u.last_name,
  COUNT(DISTINCT i.id) AS total_sessions,
  COUNT(DISTINCT m.id) AS total_messages
FROM
  users u
LEFT JOIN internet i ON u.user_id = i.user_id
LEFT JOIN messages m ON u.user_id = m.user_id
GROUP BY
  u.user_id, u.first_name, u.last_name;
```

---

# Краткие пояснения

- **JOIN** связывает таблицы и позволяет вытаскивать сразу связанный набор данных.
    - INNER — только совпадающие
    - LEFT — все из левой + совпадения из правой (или NULL)
    - RIGHT — все из правой + совпадения из левой (или NULL)
    - FULL — всё из обеих
    - CROSS — все возможные пары

- **UNION ALL** — "склеивает" результаты двух запросов, **не** убирая дубликаты.

- **GROUP BY** — группировка с агрегацией (подсчётом/суммированием и т.д.) по какому-то признаку.

- **HAVING** — фильтрация уже после группировки, по агрегатному признаку (например, HAVING COUNT(*) > 3).

---

# Задачи

---

### 1. Выведите все города, в которых хотя бы один пользователь зарегистрировался после 2023-01-01.
```sql
SELECT city
FROM users
WHERE reg_date > '2023-01-01'
GROUP BY city;
```
---

### 2. Получите список всех используемых тарифов без повторений, отсортированный по алфавиту.
```sql
SELECT DISTINCT tariff FROM users
ORDER BY tariff;
```
---

### 3. Для каждого возраста пользователей (age) — сколько пользователей этого возраста.
```sql
SELECT age, COUNT(*) AS user_count
FROM users
GROUP BY age
ORDER BY age;
```
---

### 4. Для каждого пользователя — дата первой интернет-сессии (если сессий не было, не выводить пользователя).
```sql
SELECT user_id, MIN(session_date) AS first_session
FROM internet
GROUP BY user_id;
```
---

### 5. Покажите топ-3 города по количеству активных пользователей (кто совершил хотя бы одну интернет-сессию).
```sql
SELECT u.city, COUNT(DISTINCT u.user_id) AS active_users
FROM users u
JOIN internet i ON u.user_id = i.user_id
GROUP BY u.city
ORDER BY active_users DESC
LIMIT 3;
```
---

### 6. Для каждого месяца 2024 года — суммарный интернет-трафик всех пользователей.
```sql
SELECT EXTRACT(MONTH FROM session_date) AS month, SUM(mb_used) AS total_mb
FROM internet
WHERE EXTRACT(YEAR FROM session_date) = 2024
GROUP BY month
ORDER BY month;
```
---

### 7. Вывести каждого пользователя и тариф, для которых суммарный трафик превышает 5000 МБ.
```sql
SELECT u.user_id, u.first_name, u.tariff, SUM(i.mb_used) AS total_mb
FROM users u
JOIN internet i ON u.user_id = i.user_id
GROUP BY u.user_id, u.first_name, u.tariff
HAVING SUM(i.mb_used) > 5000;
```
---

### 8. Для каждого пользователя посчитать, сколько раз он пользовался интернетом в январе любого года.
```sql
SELECT user_id, COUNT(*) AS jan_sessions
FROM internet
WHERE EXTRACT(MONTH FROM session_date) = 1
GROUP BY user_id;
```
---

### 9. Сколько уникальных пользователей отправили хотя бы одно сообщение за все годы?
```sql
SELECT COUNT(DISTINCT user_id) AS active_msg_users
FROM messages;
```
---

### 10. Для каждого тарифного плана показать средний возраст пользователей, которые ни разу не пользовались интернетом.
```sql
SELECT u.tariff, AVG(u.age) AS avg_age
FROM users u
LEFT JOIN internet i ON u.user_id = i.user_id
WHERE i.id IS NULL
GROUP BY u.tariff;
```
---

### 11. Список городов, где более 80% пользователей совершили интернет-сессию (от всех пользователей этого города).
```sql
SELECT u.city
FROM users u
LEFT JOIN internet i ON u.user_id = i.user_id
GROUP BY u.city
HAVING COUNT(i.id) * 1.0 / COUNT(u.user_id) > 0.8;
```
---

### 12. Для каждой даты — число пользователей, которые в этот день отправили хотя бы одно сообщение.
```sql
SELECT message_date, COUNT(DISTINCT user_id) AS user_count
FROM messages
GROUP BY message_date
ORDER BY message_date;
```
---

### 13. Получить для каждого города: максимальный возраст пользователя, который пользовался интернетом.
```sql
SELECT u.city, MAX(u.age) AS max_age
FROM users u
JOIN internet i ON u.user_id = i.user_id
GROUP BY u.city;
```
---

### 14. Пользователи, которые отправляли сообщения только в 2023 году и не были активны в другие годы (по сообщениям).
```sql
SELECT m.user_id
FROM messages m
GROUP BY m.user_id
HAVING MIN(EXTRACT(YEAR FROM m.message_date)) = 2023 AND MAX(EXTRACT(YEAR FROM m.message_date)) = 2023;
```
---

### 15. Для каждого дня в июне 2024 — сколько всего сообщений было отправлено.
```sql
SELECT message_date, COUNT(*) AS msg_count
FROM messages
WHERE message_date BETWEEN '2024-06-01' AND '2024-06-30'
GROUP BY message_date
ORDER BY message_date;
```
---

### 16. Для каждого тарифа — количество пользователей, которые ушли (churn_date не NULL).
```sql
SELECT tariff, COUNT(*) AS churned_users
FROM users
WHERE churn_date IS NOT NULL
GROUP BY tariff;
```
---

### 17. Выведите user_id, которые совершили хотя бы одну интернет-сессию в 2024 и хотя бы одну в 2023 году.
```sql
SELECT i1.user_id
FROM internet i1
JOIN internet i2 ON i1.user_id = i2.user_id
WHERE EXTRACT(YEAR FROM i1.session_date) = 2024 AND EXTRACT(YEAR FROM i2.session_date) = 2023
GROUP BY i1.user_id;
```
---

### 18. Для каждого пользователя укажите число дней, когда он либо пользовался интернетом, либо отправил сообщение.
```sql
SELECT u.user_id, u.first_name, COUNT(DISTINCT COALESCE(i.session_date, m.message_date)) AS active_days
FROM users u
LEFT JOIN internet i ON u.user_id = i.user_id
LEFT JOIN messages m ON u.user_id = m.user_id
GROUP BY u.user_id, u.first_name;
```
---

### 19. Посчитать количество пользователей для каждого сочетания возраста и города.
```sql
SELECT city, age, COUNT(*) AS user_count
FROM users
GROUP BY city, age
ORDER BY city, age;
```
---

### 20. Для всех пользователей вывести разницу между датой регистрации и датой первой интернет-сессии (если есть сессии).
```sql
SELECT u.user_id, u.first_name, u.last_name, MIN(i.session_date) - u.reg_date AS days_to_first_session
FROM users u
JOIN internet i ON u.user_id = i.user_id
GROUP BY u.user_id, u.first_name, u.last_name, u.reg_date;
```
---

### 21. Для каждого месяца 2024 года посчитать уникальных пользователей с сессиями и уникальных с сообщениями.
```sql
SELECT 
  EXTRACT(MONTH FROM session_date) AS month, 
  COUNT(DISTINCT user_id) AS internet_users,
  0 AS message_users
FROM internet
WHERE EXTRACT(YEAR FROM session_date) = 2024
GROUP BY month

UNION ALL

SELECT 
  EXTRACT(MONTH FROM message_date) AS month, 
  0 AS internet_users,
  COUNT(DISTINCT user_id) AS message_users
FROM messages
WHERE EXTRACT(YEAR FROM message_date) = 2024
GROUP BY month;
```
---

### 22. Для каждого города — сколько пользователей никогда не отправляли сообщения.
```sql
SELECT u.city, COUNT(u.user_id) AS never_sent_msg
FROM users u
LEFT JOIN messages m ON u.user_id = m.user_id
WHERE m.id IS NULL
GROUP BY u.city;
```
---

### 23. Тарифы, где средний возраст пользователей — чётное число лет (результат округленный до целого).
```sql
SELECT tariff, ROUND(AVG(age)) AS avg_age
FROM users
GROUP BY tariff
HAVING ROUND(AVG(age)) % 2 = 0;
```
---

### 24. Для каждой интернет-сессии вывести пользователя (имя, фамилию) и дату.
```sql
SELECT u.first_name, u.last_name, i.session_date
FROM internet i
JOIN users u ON i.user_id = u.user_id;
```
---

### 25. Для каждого города посчитать, сколько интернет-сессий пришлось на одного пользователя (среднее).
```sql
SELECT u.city, COUNT(i.id) * 1.0 / COUNT(DISTINCT u.user_id) AS avg_sess_per_user
FROM users u
LEFT JOIN internet i ON u.user_id = i.user_id
GROUP BY u.city;
```
---

### 26. Вывести даты, когда хотя бы 5 пользователей из Москвы пользовались интернетом.
```sql
SELECT i.session_date
FROM internet i
JOIN users u ON i.user_id = u.user_id
WHERE u.city = 'Москва'
GROUP BY i.session_date
HAVING COUNT(DISTINCT i.user_id) >= 5;
```
---

### 27. Для каждого тарифа — максимальное число сообщений, отправленных одним пользователем тарифа.
```sql
SELECT u.tariff, MAX(msg_count) AS max_msgs
FROM (
  SELECT u.tariff, u.user_id, COUNT(m.id) AS msg_count
  FROM users u
  LEFT JOIN messages m ON u.user_id = m.user_id
  GROUP BY u.tariff, u.user_id
) AS sub
GROUP BY sub.tariff;
```
---

### 28. Посчитать совокупное количество событий (интернет-сессии + сообщения) по каждому пользователю.
```sql
SELECT u.user_id, u.first_name, COUNT(i.id) + COUNT(m.id) AS total_events
FROM users u
LEFT JOIN internet i ON u.user_id = i.user_id
LEFT JOIN messages m ON u.user_id = m.user_id
GROUP BY u.user_id, u.first_name;
```
---

### 29. Города, где хотя бы один пользователь уходил (churn_date есть), а средний возраст превышает 30.
```sql
SELECT city, AVG(age) AS avg_age
FROM users
WHERE churn_date IS NOT NULL
GROUP BY city
HAVING AVG(age) > 30;
```
---

### 30. Вывести топ-3 дня с максимальным количеством суммарных событий (сессии + сообщения).
```sql
SELECT day, SUM(events) AS total_events
FROM (
  SELECT session_date AS day, COUNT(*) AS events
  FROM internet
  GROUP BY session_date
  UNION ALL
  SELECT message_date AS day, COUNT(*) AS events
  FROM messages
  GROUP BY message_date
) all_events
GROUP BY day
ORDER BY total_events DESC
LIMIT 3;
```
---